In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment',  None)

In [2]:
customer = pd.read_csv('customer_join.csv')

In [3]:
uselog_months = pd.read_csv('use_log_months.csv')

In [4]:
year_months = list(uselog_months['연월'].unique())

In [5]:
uselog = pd.DataFrame()

In [6]:
uselog_months.head()

연월 customer_id  count
0  201804    AS002855      4
1  201804    AS009013      2
2  201804    AS009373      3
3  201804    AS015315      6
4  201804    AS015739      7

In [7]:
for i in range (1,len(year_months)) :
    tmp = uselog_months.loc[uselog_months['연월'] == year_months[i]]
    tmp.rename(columns = {'count' : 'count_0'},inplace = True)
    tmp_before = uselog_months.loc[uselog_months['연월'] == year_months[i-1]]
    del tmp_before['연월']
    tmp_before.rename(columns = {'count' : 'count_1'},inplace = True)
    tmp = pd.merge(tmp, tmp_before, on = 'customer_id', how = 'left')
    uselog = pd.concat([uselog,tmp],ignore_index = True)
uselog.head()

연월 customer_id  count_0  count_1
0  201805    AS002855        5      4.0
1  201805    AS009373        4      3.0
2  201805    AS015233        7      NaN
3  201805    AS015315        3      6.0
4  201805    AS015739        5      7.0

In [8]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer['is_deleted']==1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)) :
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i]-relativedelta(months = 1)

In [9]:
exit_customer['연월'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime('%Y%m')

In [10]:
uselog['연월'] = uselog['연월'].astype(str)

In [11]:
exit_uselog = pd.merge(uselog,exit_customer, on = ['customer_id','연월'], how = 'left')

In [12]:
len(uselog)

33851

In [13]:
exit_uselog.head()

연월 customer_id  count_0  count_1 name class gender start_date end_date  \
0  201805    AS002855        5      4.0  NaN   NaN    NaN        NaN      NaT   
1  201805    AS009373        4      3.0  NaN   NaN    NaN        NaN      NaT   
2  201805    AS015233        7      NaN  NaN   NaN    NaN        NaN      NaT   
3  201805    AS015315        3      6.0  NaN   NaN    NaN        NaN      NaT   
4  201805    AS015739        5      7.0  NaN   NaN    NaN        NaN      NaT   

  campaign_id  ...  price campaign_name  mean median  max  min  routine_flg  \
0         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   
1         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   
2         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   
3         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   
4         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   

   calc_date  membership_period exit_date  
0        NaN                NaN       NaN  
1        NaN                NaN       NaN  
2        NaN                NaN       NaN  
3        NaN                NaN       NaN  
4        NaN                NaN       NaN  

[5 rows x 22 columns]

In [14]:
exit_uselog = exit_uselog.dropna(subset = ['name'])

In [15]:
len(exit_uselog)

1104

In [16]:
len(exit_uselog['customer_id'].unique())

1104

In [17]:
exit_uselog.head()

연월 customer_id  count_0  count_1   name class gender  start_date  \
19   201805    AS055680        3      3.0  XXXXX   C01      M  2018-03-01   
57   201805    AS169823        2      3.0     XX   C01      M  2017-11-01   
110  201805    AS305860        5      3.0   XXXX   C01      M  2017-06-01   
128  201805    AS363699        5      3.0  XXXXX   C01      M  2018-02-01   
147  201805    AS417696        1      4.0     XX   C03      F  2017-09-01   

      end_date campaign_id  ...    price campaign_name      mean median  max  \
19  2018-06-30         CA1  ...  10500.0          2_일반  3.000000    3.0  3.0   
57  2018-06-30         CA1  ...  10500.0          2_일반  3.000000    3.0  4.0   
110 2018-06-30         CA1  ...  10500.0          2_일반  3.333333    3.0  5.0   
128 2018-06-30         CA1  ...  10500.0          2_일반  3.333333    3.0  5.0   
147 2018-06-30         CA1  ...   6000.0          2_일반  2.000000    1.0  4.0   

     min  routine_flg   calc_date  membership_period            exit_date  
19   3.0          0.0  2018-06-30                3.0  2018-05-30 00:00:00  
57   2.0          1.0  2018-06-30                7.0  2018-05-30 00:00:00  
110  2.0          0.0  2018-06-30               12.0  2018-05-30 00:00:00  
128  2.0          0.0  2018-06-30                4.0  2018-05-30 00:00:00  
147  1.0          0.0  2018-06-30                9.0  2018-05-30 00:00:00  

[5 rows x 22 columns]

### 지속 회원의 데이터를 작성하자

In [18]:
conti_customer = customer.loc[customer['is_deleted']==0]

In [19]:
conti_uselog = pd.merge(uselog,conti_customer,how = 'left', on = 'customer_id')

In [20]:
len(conti_uselog)

33851

In [21]:
conti_uselog = conti_uselog.dropna(subset=['name'])

In [22]:
len(conti_uselog)

27422

In [23]:
conti_uselog = conti_uselog.sample(frac = 1).reset_index(drop = True)
conti_uselog = conti_uselog.drop_duplicates(subset = 'customer_id')

In [24]:
len(conti_uselog)

2842

#### 지속 회원 데이터와 탈퇴 회원 데이터 결합

In [25]:
predict_data = pd.concat([conti_uselog,exit_uselog], ignore_index = True)

In [26]:
len(predict_data)

3946

In [27]:
predict_data['period'] = 0

In [28]:
predict_data.head()

연월 customer_id  count_0  count_1   name class gender  start_date  \
0  201903    AS458614        4      4.0    XXX   C01      F  2017-03-01   
1  201807    PL343970        4      6.0  XXXXX   C01      F  2015-10-01   
2  201808    OA436769        7      NaN   XXXX   C03      M  2018-08-03   
3  201901    HI778223        3      6.0     XX   C03      M  2015-07-01   
4  201811    IK149763        5      3.0  XXXXX   C03      M  2017-09-01   

  end_date campaign_id  ...  campaign_name      mean  median   max  min  \
0      NaN         CA1  ...           2_일반  5.333333     4.5  10.0  3.0   
1      NaN         CA1  ...           2_일반  5.250000     5.0   7.0  4.0   
2      NaN         CA1  ...           2_일반  7.000000     7.0   9.0  4.0   
3      NaN         CA1  ...           2_일반  4.250000     4.5   7.0  2.0   
4      NaN         CA1  ...           2_일반  5.250000     5.0   8.0  2.0   

   routine_flg   calc_date  membership_period  exit_date period  
0          1.0  2019-04-30               25.0        NaN      0  
1          1.0  2019-04-30               42.0        NaN      0  
2          1.0  2019-04-30                8.0        NaN      0  
3          1.0  2019-04-30               45.0        NaN      0  
4          1.0  2019-04-30               19.0        NaN      0  

[5 rows x 23 columns]

In [29]:
predict_data['now_date'] = pd.to_datetime(predict_data['연월'],format = '%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

In [30]:
for i in range(len(predict_data)) :
    delta = relativedelta(predict_data['now_date'][i],predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years *12 + delta.months)

In [31]:
predict_data.head()

연월 customer_id  count_0  count_1   name class gender start_date  \
0  201903    AS458614        4      4.0    XXX   C01      F 2017-03-01   
1  201807    PL343970        4      6.0  XXXXX   C01      F 2015-10-01   
2  201808    OA436769        7      NaN   XXXX   C03      M 2018-08-03   
3  201901    HI778223        3      6.0     XX   C03      M 2015-07-01   
4  201811    IK149763        5      3.0  XXXXX   C03      M 2017-09-01   

  end_date campaign_id  ...      mean median   max  min  routine_flg  \
0      NaN         CA1  ...  5.333333    4.5  10.0  3.0          1.0   
1      NaN         CA1  ...  5.250000    5.0   7.0  4.0          1.0   
2      NaN         CA1  ...  7.000000    7.0   9.0  4.0          1.0   
3      NaN         CA1  ...  4.250000    4.5   7.0  2.0          1.0   
4      NaN         CA1  ...  5.250000    5.0   8.0  2.0          1.0   

    calc_date  membership_period  exit_date  period   now_date  
0  2019-04-30               25.0        NaN      24 2019-03-01  
1  2019-04-30               42.0        NaN      33 2018-07-01  
2  2019-04-30                8.0        NaN       0 2018-08-01  
3  2019-04-30               45.0        NaN      42 2019-01-01  
4  2019-04-30               19.0        NaN      14 2018-11-01  

[5 rows x 24 columns]

In [32]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               251
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [33]:
predict_data = predict_data.dropna(subset = ['count_1'])

In [34]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2643
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2643
period                  0
now_date                0
dtype: int64

### 문자열 변수를 처리할 수 있게 가공하자

In [35]:
target_col = ['campaign_name','class_name','gender','count_1','routine_flg','period','is_deleted']

In [36]:
predict_data = predict_data[target_col]

In [37]:
predict_data.head()

campaign_name class_name gender  count_1  routine_flg  period  is_deleted
0          2_일반       0_종일      F      4.0          1.0      24         0.0
1          2_일반       0_종일      F      6.0          1.0      33         0.0
3          2_일반       2_야간      M      6.0          1.0      42         0.0
4          2_일반       2_야간      M      3.0          1.0      14         0.0
5          2_일반       0_종일      F      8.0          1.0      12         0.0

##### 더미변수 만들기

In [38]:
predict_data = pd.get_dummies(predict_data)

In [39]:
predict_data.head()

count_1  routine_flg  period  is_deleted  campaign_name_0_입회비반액할인  \
0      4.0          1.0      24         0.0                        0   
1      6.0          1.0      33         0.0                        0   
3      6.0          1.0      42         0.0                        0   
4      3.0          1.0      14         0.0                        0   
5      8.0          1.0      12         0.0                        0   

   campaign_name_1_입회비무료  campaign_name_2_일반  class_name_0_종일  \
0                      0                   1                1   
1                      0                   1                1   
3                      0                   1                0   
4                      0                   1                0   
5                      0                   1                1   

   class_name_1_주간  class_name_2_야간  gender_F  gender_M  
0                0                0         1         0  
1                0                0         1         0  
3                0                1         0         1  
4                0                1         0         1  
5                0                0         1         0

In [40]:
del predict_data['campaign_name_2_일반']

In [41]:
del predict_data['class_name_2_야간']

In [42]:
del predict_data['gender_M']

In [43]:
predict_data.head()

count_1  routine_flg  period  is_deleted  campaign_name_0_입회비반액할인  \
0      4.0          1.0      24         0.0                        0   
1      6.0          1.0      33         0.0                        0   
3      6.0          1.0      42         0.0                        0   
4      3.0          1.0      14         0.0                        0   
5      8.0          1.0      12         0.0                        0   

   campaign_name_1_입회비무료  class_name_0_종일  class_name_1_주간  gender_F  
0                      0                1                0         1  
1                      0                1                0         1  
3                      0                0                0         0  
4                      0                0                0         0  
5                      0                1                0         1

### 의사결정 트리를 사용해서 탈퇴 예측 모델을 구축하자

In [44]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

In [45]:
exi = predict_data.loc[predict_data['is_deleted']==1]

In [46]:
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exi))

In [47]:
X = pd.concat([exi,conti],ignore_index = True )

In [48]:
y = X['is_deleted']

In [49]:
del X['is_deleted']

In [50]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

In [51]:
model = DecisionTreeClassifier(random_state=0)

In [52]:
model.fit(X_train,y_train)

DecisionTreeClassifier(random_state=0)

In [53]:
y_test_pred = model.predict(X_test)

In [54]:
print(y_test_pred)

[0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0.
 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0.
 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.

In [55]:
results_test = pd.DataFrame({'y_test' : y_test,
                             'y_pred' : y_test_pred})

In [56]:
results_test

y_test  y_pred
1733     0.0     0.0
1746     0.0     0.0
1776     0.0     0.0
1510     0.0     0.0
256      1.0     1.0
...      ...     ...
1888     0.0     0.0
2011     0.0     0.0
1050     1.0     1.0
1686     0.0     0.0
1112     0.0     0.0

[526 rows x 2 columns]

In [57]:
correct = len(results_test.loc[results_test['y_test'] == results_test['y_pred']])

In [58]:
data_count = len(results_test)

In [59]:
score_test = correct/data_count

In [60]:
score_test

0.8859315589353612

In [61]:
model.score(X_test,y_test)

0.8859315589353612

In [62]:
model.score(X_train,y_train)

0.9778200253485425

In [63]:
X = pd.concat([exi,conti],ignore_index = True)

In [64]:
y=X['is_deleted']

In [65]:
del X['is_deleted']

In [66]:
X_train, X_test , y_train,y_test = sklearn.model_selection.train_test_split(X,y)

In [67]:
model = DecisionTreeClassifier(random_state = 0 , max_depth = 5)

In [68]:
model.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=5, random_state=0)

In [69]:
model.score(X_test,y_test)

0.9201520912547528

In [70]:
model.score(X_train,y_train)

0.9296577946768061

In [71]:
importance = pd.DataFrame({'feature_names' : X.columns, 
                           'coefficient' : model.feature_importances_})

In [72]:
importance

feature_names  coefficient
0                  count_1     0.325727
1              routine_flg     0.148567
2                   period     0.525486
3  campaign_name_0_입회비반액할인     0.000000
4    campaign_name_1_입회비무료     0.000220
5          class_name_0_종일     0.000000
6          class_name_1_주간     0.000000
7                 gender_F     0.000000

### 회원탈퇴를 예측하자

In [74]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = '입회비무료'
class_name = '종일'
gender = 'M'

In [76]:
if campaign_name == '입회비반값할인':
    campaign_name_list = [1,0]
elif campaign_name == '입회비무료' :
    campaign_name_list = [0,1]
elif campaign_name == '일반' :
    campaign_name_list = [0,0]
if class_name == '종일' :
    class_name_list = [1,0]
elif class_name == '주간':
    class_name_list = [0,1]
elif class_name == '야간' :
    class_name_list = [0,0]
if gender == 'F' :
    gender_list = [1]
elif gender == 'M' :
    gender_list = [0]
input_data = [count_1,routine_flg,period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [77]:
model.predict([input_data])

C:\Users\82105\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1.])

In [78]:
model.predict_proba([input_data])

C:\Users\82105\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([[0.1, 0.9]])